In [16]:
import os 
import pandas as pd
import numpy as np
from pydub import AudioSegment

import IPython.display as ipd

import librosa
import librosa.display
import librosa.feature
import matplotlib.pyplot as plt

import utils
import processing

In [17]:
audio_folder = '../../data/ICBHI_final_database'

In [18]:
list_feature =  []
list_files = []
for file in os.listdir(audio_folder):
    if file.endswith('.wav'):
        audio_file = file.split('.wav')[0]
        audio_data_file, audio_event_file = processing.load_file_from_recording_name(audio_folder,audio_file)
        data_filtered = processing.preprocess_data(audio_data_file, audio_event_file)
        features, list_feature_type = processing.extract_features(data_filtered)
        list_files.append(audio_file)
        list_feature.append(features)

In [19]:
features_m = processing.prepare_features_ml(list_feature,mean_data=True)
features_m.shape

(920, 77)

In [20]:
features = processing.prepare_features_ml(list_feature,mean_data=False)
features.shape

(920, 77, 674)

In [52]:
df_info = pd.read_csv('../../data/info_dataset.csv')
df_info=df_info[df_info.filename.isin(list_files)]

In [55]:
df_info = df_info.set_index('filename')

In [56]:
data_dict = dict()
data_dict['feature_mean'] = features_m
data_dict['features'] = features
data_dict['info_feature_type'] = list_feature_type
data_dict['label'] =df_info.reindex(list_files).Outcome.to_numpy()

In [60]:
import pickle 
with open('../../data/data_model/data_dict.pkl','wb') as f:
    pickle.dump(data_dict,f)
    